<a href="https://colab.research.google.com/github/voodoohop/pollinations/blob/master/colabs/pollinator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Code (Frontend and IPFS Connection)

In [17]:
#@title Install IPFS

#!pip install aioipfs nest-asyncio coolname
!pip install asyncio nest-asyncio coolname aioipfs
#!pip install ipfshttpclient
#!npm install -g nodemon


In [18]:
#@title Config
from coolname import generate_slug
ipfs_nodeid = generate_slug(2)#asyncio.run(loop.create_task(get_node_id()))
IPFS_PEER = "18.157.173.110"
ipfs_root = "/content/ipfs"

print(f"---Settings---\nNODE_ID: {ipfs_nodeid}\nIPFS_ROOT: {ipfs_root}\nIPFS_PEER: {IPFS_PEER}")

---Settings---
NODE_ID: strict-butterfly
IPFS_ROOT: /content/ipfs
IPFS_PEER: 18.157.173.110


In [19]:
#@title Empty and create ipfs root: */content/ipfs*



!rm -r $ipfs_root
!mkdir -p $ipfs_root/input
!mkdir -p $ipfs_root/output
!echo "Created IPFS resource" > $ipfs_root/output/log

parameters={"bla":"hello"}
for key, default in parameters.items():
    print(default, file=open(f"{ipfs_root}/input/{key}", 'a'))



In [ ]:
#@title Connect to local IPFS node and publish ipfs root
import sys
import json

import nest_asyncio
nest_asyncio.apply()

import asyncio
import aioipfs


def await_async(x):
  #asyncio.ensure_future(x)
  asyncio.run(x)


async def asyncfunc():
  await asyncio.sleep(5)

#asyncio.run(loop.create_task(sub("hello")))

print("slept", await_async(asyncfunc()))





In [20]:

import aioipfs



#client = aioipfs.AsyncIPFS(debug=True)
clientRemote = aioipfs.AsyncIPFS(host=IPFS_PEER,port=5002,debug=True)
client=clientRemote
ipfs_nodeid = None
ipfs_contentid = None

async def get_node_id():
    return (await client.id())["ID"]

async def _add_and_publish(path=ipfs_root, pin_remote=False):
    global ipfs_contentid
    last_added=None
    async for added_file in client.add(path, recursive=True):
      print('Imported file', added_file)
      last_added=added_file["Hash"]
    ipfs_contentid = last_added
    update_frontend()
    return last_added

def add_and_publish():
  await_async(_add_and_publish())

#loop = asyncio.get_event_loop()

In [ ]:
#@title Connect to IPFS Pubsub

async def subscribe(topic, handler=None):
  print("subscribed to",topic)
  async for message in client.pubsub.sub(topic):
    if handler:
      await handler(message)
    else:
      print("No handler. Received IPFS pub message",message)
    break

async def publish(topic, data):
  print("publishing",topic,data)
  await client.pubsub.pub(topic, data)

def on_frontend_data(handler):
  async def handle(ipfs_message):
    await handler(ipfs_message["data"])
  await_async(subscribe(ipfs_nodeid, handle))

def update_frontend():
  await_async(publish(ipfs_nodeid, ipfs_contentid))



add_and_publish()

In [21]:
#@title Create connection to frontend UI

import IPython
from google.colab import output

display(IPython.display.Javascript(f'''
  const iframe = document.createElement("iframe")
  iframe.src = `http://localhost:3000/incolab.html?node={ipfs_nodeid},{ipfs_contentid}`

  document.querySelector("#output-area").appendChild(iframe)
'''))



<IPython.core.display.Javascript object>

In [ ]:
async def update_ipfs_content(contentid):
  global ipfs_contentid
  contentid = contentid.decode("utf-8")
  
  await client.get(contentid)
  !cp -rv $contentid/* $ipfs_root
  #!rm -r $contentid
  ipfs_contentid = contentid

on_frontend_data(update_ipfs_content)


In [22]:
from glob import glob
from os.path import basename
for path in glob(f"{ipfs_root}/input/*"):
  key = basename(path)
  with open(path,'r') as f:
    value = f.read()
    parameters[key] = value

print("New parameters", parameters)

New parameters {'bla': 'hello\n'}


## DISABLED! CODE (DEEP-DAZE)


DISABLED

```
from subprocess import Popen, PIPE, STDOUT
from sys import stdout
from glob import glob
import os
import json
import itertools
from base64 import b64encode
from time import sleep
import requests
import os
from mimetypes import guess_type 
import re
flatten = itertools.chain.from_iterable

#parameters["text"] = "A black and white photo of Saeko with her new Elektron FM synthesizer"
def run(text, args, watch_media_paths=["./*.jpg","./*.mp4"]):
  print("Running",text)
  for to_remove in watch_media_paths:
    for filepath in glob(to_remove):
      print("Deleting",filepath,"before starting.")
      os.remove(filepath)

  arguments = args.split(" ")
  cmd = ['imagine'] + arguments +[text]
  
  p = Popen(cmd, stdout = PIPE, 
          stderr = STDOUT, shell = True)

  found_media_files=[]
  
  while True:
    line = p.stdout.readline()#
    if not line: break

    stdout.write(line)
    with open(f"{ipfs_root}/log","w") as log_out:
      log_out.write(line.decode("utf-8"))
      log_out.flush()
    #headers = { 
    #    "params": params,
    #    "text": text
    #}
    #conn.send(body=line.decode("utf-8"), headers=headers, destination='/topic/colabOut')
    #print("new contentid", client.add(ipfs_root, recursive=True))
    
    #continue
    new_found_media_files = get_matching_files(watch_media_paths)
    new_found_media_files.sort(key=os.path.getmtime)
    added_media_files = list(set(new_found_media_files)-set(found_media_files))
    found_media_files = new_found_media_files
    pin=False
    if len(added_media_files) > 0:
      for added in added_media_files:
        sleep(0.1)
        if len(added) > 0:
          !cp $added $ipfs_root/output
          print("Adding:", added,"to ipfs root")
          pin=True
    add_and_publish()    
       

def get_matching_files(search_paths):
  return list(flatten([glob(search_path) for search_path in search_paths]))

run(**parameters)

sleep(1000000)
```



# New section

In [ ]:

import json
def notebook_add_metadata(target_notebook):

  with open(target_notebook, "r") as f:
    j = json.load(f)

  param_cell =1
  j['cells'][param_cell]['metadata']['tags'] = ['parameters']
  j['cells'][param_cell]['metadata']
  j['metadata']['kernelspec']['language'] = 'python'

  with open(target_notebook, "w") as f:
    print("Writing modified notebook to", target_notebook)
    json.dump(j, f)

!pip install gdown
!gdown --id 1zOA0bUJ68zuZXcbCoN2LPNjnkvxzFpv_ -O latentvisions.ipynb

notebook_add_metadata("./latentvisions.ipynb")


In [ ]:
#!pip install --upgrade pip
!pip install papermill
#!pip install --upgrade ipython
#import papermill as pm
from subprocess import Popen, PIPE, STDOUT
from sys import stdout
import os

#parameters["text"] = "A black and white photo of Saeko with her new Elektron FM synthesizer"
def run(parameters, watch_media_paths="./*.jpg"):
  print("Running",parameters)
 
  for filepath in glob(watch_media_paths):
    print("Deleting",filepath,"before starting.")
    os.remove(filepath)

  _params = [["-p", key, value] for key, value in parameters.items()]
  params = [str(val) for sublist in _params for val in sublist]
  #%run latentvisions.ipynb
  #!papermill "./latentvisions.ipynb" ./output 
  cmd = ["papermill", "/content/latentvisions.ipynb","/content/latentvisions_output.ipynb"] + params
  print(" ".join(cmd))
  p = Popen(cmd, stdout = PIPE, 
          stderr = STDOUT, shell = False)

  found_media_files=[]

  while True:
    line = p.stdout.readline()#
    if not line: break

    stdout.write(line)
    with open(f"{ipfs_root}/log","w") as log_out:
      log_out.write(line.decode("utf-8"))
      log_out.flush()
      
    #headers = { 
    #    "params": params,
    #    "text": text
    #}
    #conn.send(body=line.decode("utf-8"), headers=headers, destination='/topic/colabOut')
    #print("new contentid", client.add(ipfs_root, recursive=True))
    
    #continue
    new_found_media_files = glob(watch_media_paths)
    new_found_media_files.sort(key=os.path.getmtime)
    added_media_files = list(set(new_found_media_files)-set(found_media_files))
    found_media_files = new_found_media_files
    pin=False
    if len(added_media_files) > 0:
      for added in added_media_files:
        sleep(0.1)
        if len(added) > 0:
          !cp $added $ipfs_root/output
          print("Adding:", added,"to ipfs root")
          pin=True
    add_and_publish()    
        
while True:
  run(parameters,"./taming_transformers/*.jpg")